# Agent 2: Cyber Collector

In this multi-agent system, **`Cyber Collector`** serves as the first agent for information collection and organization, with its main task being to provide high-quality, structured information input for subsequent agents.

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

/Users/terry_chou/Documents/trend-ai-workshop/TrendMicro_workshop_v2/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Define knowledge base

In [3]:
# Upload file to knowledge base, and it will automatically process, vectorize and store
file = project_client.agents.upload_file_and_poll(
    file_path="./data/cyber_security_framework.md", purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

# create vector store
vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-wWghPbtythur1PWODo3bPW5T
Created vector store, vector store ID: vs_gHUxCSQ5tvdaFn30ccTe3o3J


## Create Agent

In [4]:
from azure.ai.projects.models import (
    FileSearchTool,
)

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

cyber_collector = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="cyber_collector",
    description="The cyber collector can collect information about cyber security from its knowledge base.",
    instructions="Hello, you are helpful assistant and can search information from uploaded files.",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {cyber_collector.id}")

Created agent, agent ID: asst_dpe1ZraNLS5pl6zNvAlc5JdG


In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Collect the latest cybersecurity threat information relevant to TrendMicro as well as their current metrics."
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=cyber_collector.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Tokens per Minute Rate Limit need to be taken care
    print(f"Run failed: {run.last_error}")

In [ ]:
# Display all messages in the thread
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

In [ ]:
# Print citations from the messages
for citation in messages.file_citation_annotations:
    print(citation)

In [ ]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
message_data = project_client.agents.list_messages(thread_id=thread.id).data
for msg in message_data:
    print(msg.content)

In [ ]:
# Delete the file when done
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_file(file_id=file.id)
print("Deleted file")

# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")